<a href="https://colab.research.google.com/github/ARJUN108-verma/LangGraph/blob/main/6_Reducer_Functions_in_actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reducer Functions in actions:-

### Set the OpenAI API Key as an Environment Variable

In [ ]:
!pip install mypy_ipython

In [ ]:
%load_ext dotenv
%dotenv
%load_ext mypy_ipython

### Import Relevant Classes and Functions:-

In [ ]:
from langgraph.graph import START, END, StateGraph, add_messages
from typing_extensions import TypedDict
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from collections.abc import Sequence
from typing import Literal, Annotated

### Get Familiar with the add_messages Function:-

In [ ]:
my_list = add_messages([HumanMessage("Hi! I'm Oscar."),
                        AIMessage("Hey, Oscar. How can I assist you?")],
                       [HumanMessage("Could you summarize today's news?")])

In [ ]:
my_list

### Define the State:-

In [ ]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

### Define the Nodes

In [ ]:
chat = ChatOpenAI(model = "gpt-4o",
                  seed = 365,
                  temperature = 0,
                  max_completion_tokens = 100)

In [ ]:
def ask_question(state: State) -> State:

    print(f"\n-------> ENTERING ask_question:")
    for i in state["messages"]:
        i.pretty_print()

    question = "What is your question?"
    print(question)

    return State(messages = [AIMessage(question), HumanMessage(input())])

In [ ]:
def chatbot(state: State) -> State:

    print(f"\n-------> ENTERING chatbot:")
    for i in state["messages"]:
        i.pretty_print()

    response = chat.invoke(state["messages"])
    response.pretty_print()

    return State(messages = [response])

In [ ]:
def ask_another_question(state: State) -> State:

    print(f"\n-------> ENTERING ask_another_question:")
    for i in state["messages"]:
        i.pretty_print()

    question = "Would you like to ask one more question (yes/no)?"
    print(question)

    return State(messages = [AIMessage(question), HumanMessage(input())])

### Define the Routing Function:-

In [ ]:
def routing_function(state: State) -> Literal["ask_question", "__end__"]:

    if state["messages"][-1].content == "yes":
        return "ask_question"
    else:
        return "__end__"

### Define the Graph:-

In [ ]:
graph = StateGraph(State)

In [ ]:
graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("ask_another_question", ask_another_question)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "ask_another_question")
graph.add_conditional_edges(source = "ask_another_question",
                            path = routing_function)

In [ ]:
graph_compiled = graph.compile()

In [ ]:
graph_compiled

### Test the Graph:-

In [ ]:
graph_compiled.invoke(State(messages = []))